In [1]:
import psycopg2
import pandas as pd
import numpy as np

# Creating a connection between psycopg2 and the postgresql database
conn = psycopg2.connect(database='Grand Slam Tennis Data', user='postgres', 
                        password='Letsgorangers24', host="127.0.0.1",
                       port="5432")

if conn:
    print('Connection established')

Connection established


In [2]:
# Creating a cursor object
curs = conn.cursor()

# Gathering all data from matches table
curs.execute('SELECT * FROM matches')

# Fetching the query results and storing in the variable matches
matches = curs.fetchall()

# Gathering the column names for the matches table
matches_colnames = [desc[0] for desc in curs.description]

In [3]:
# Gathering all data from points table
curs.execute('SELECT * FROM points')

# Fetching the query results and storing in the variable points
points = curs.fetchall()

# Gathering the column names for the points table
points_colnames = [desc[0] for desc in curs.description]

#curs.execute('SELECT * FROM matches LEFT JOIN points on matches.match_id = points.match_id\
#WHERE ')

#raw_data_colnames = [desc[0] for desc in curs.description]

In [4]:
# Putting all the data gathered from postgresql into towo different dataframes
matches = pd.DataFrame(matches, columns=matches_colnames)
points = pd.DataFrame(points, columns=points_colnames)

raw_data = matches.merge(points, how='outer')

Now that I have the raw data in a dataframe, I want to first clean the data, and then look at the points for the big 3  - Roger Federer, Rafael Nadal, and Novak Djokovic. I'm curious to see how the majority of their points have changed or remained over time.

In [5]:
# Setting the amount of columns to display to 100 so I can see the entire dataframe
pd.set_option("display.max_columns", 100)
pd.get_option("display.max_columns")

# Setting the amount of rows to display at default to 50
pd.set_option('display.max_rows', 50)

In [6]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1473104 entries, 0 to 1473103
Data columns (total 80 columns):
match_id                 1473104 non-null object
match_year               1302249 non-null object
slam                     1302249 non-null object
match_num                1302249 non-null object
player_1                 1300146 non-null object
player_2                 1301094 non-null object
match_status             83017 non-null object
match_winner             83017 non-null object
event_type               83017 non-null object
round                    83017 non-null object
court_name               83017 non-null object
court_id                 83017 non-null object
player_1_id              83017 non-null object
player_2_is              83017 non-null object
player_1_nation          315464 non-null object
player_2_nation          315464 non-null object
elapsed_time             1131392 non-null object
set_number               1472464 non-null object
p1_games_won           

In [7]:
# Creating a new dataframe called data which will be a copy of raw data. This is the dataset I will clean up.
data = raw_data.copy()

In [8]:
data.head()

,match_id,match_year,slam,match_num,player_1,player_2,match_status,match_winner,event_type,round,court_name,court_id,player_1_id,player_2_is,player_1_nation,player_2_nation,elapsed_time,set_number,p1_games_won,p2_games_won,set_winner,game_nuber,game_winner,point_number,point_winner,point_server,serve_speed_kmh,rally,p1_score,p2_score,p1_momentum,p2_momentum,p1_points_won,p2_points_won,p1_ace,p2_ace,p1_shot_winner,p2_shot_winner,p1_double_fault,p2_double_fault,p1_unforced_error,p2_unforced_error,p1_net_point,p2_net_point,p1_net_point_won,p2_net_point_won,p1_break_point,p2_break_point,p1_break_point_won,p2_break_point_won,p1_first_serve_in,p2_first_serve_in,p1_first_serve_won,p2_first_serve_won,p1_second_serve_in,p2_second_serve_in,p1_second_serve_won,p2_second_serve_won,p1_forced_error,p2_forced_error,history,speed_mph,p1_break_point_missed,p2_break_point_missed,serve_indicator,serve_direction,winner_fh,winner_bh,serving_to,p1_turning_point,p2_turning_point,serve_number,winner_type,winner_shot_type,p1_distance_run,p2_distance_run,rallycount,servewidth,servedepth,returndepth
0,2011-frenchopen-1101,2011,frenchopen,1101,Rafael Nadal,John Isner,Complete,1,event_MS,round_1,Court Philippe Chatrier,A,atpn409,atpi186,ESP,USA,2:11:27,3.0,4.0,3.0,0.0,7.0,1.0,165,1.0,1.0,182.0,1.0,0,0,145.0,104.0,88.0,77.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,21032.0,113.0,0.0,0.0,2.0,1.0,0.0,0.0,None,7,0,None,None,None,None,None,None,None,None,None
1,2011-frenchopen-1101,2011,frenchopen,1101,Rafael Nadal,John Isner,Complete,1,event_MS,round_1,Court Philippe Chatrier,A,atpn409,atpi186,ESP,USA,2:13:09,3.0,4.0,3.0,0.0,8.0,0.0,166,2.0,2.0,206.0,3.0,0,15,145.0,107.0,88.0,78.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,22410.0,128.0,0.0,0.0,2.0,2.0,0.0,1.0,None,0,0,None,None,None,None,None,None,None,None,None
2,2011-frenchopen-1101,2011,frenchopen,1101,Rafael Nadal,John Isner,Complete,1,event_MS,round_1,Court Philippe Chatrier,A,atpn409,atpi186,ESP,USA,2:13:37,3.0,4.0,3.0,0.0,8.0,0.0,167,1.0,2.0,188.0,2.0,15,15,147.0,106.0,89.0,78.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,21030.0,116.0,0.0,0.0,2.0,3.0,0.0,0.0,None,0,1,None,None,None,None,None,None,None,None,None
3,2011-frenchopen-1101,2011,frenchopen,1101,Rafael Nadal,John Isner,Complete,1,event_MS,round_1,Court Philippe Chatrier,A,atpn409,atpi186,ESP,USA,2:15:55,3.0,4.0,4.0,0.0,8.0,2.0,171,2.0,2.0,163.0,1.0,0,0,148.0,116.0,90.0,81.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,21412.0,101.0,0.0,0.0,1.0,3.0,1.0,0.0,None,0,12,None,None,None,None,None,None,None,None,None
4,2011-frenchopen-1101,2011,frenchopen,1101,Rafael Nadal,John Isner,Complete,1,event_MS,round_1,Court Philippe Chatrier,A,atpn409,atpi186,ESP,USA,2:14:48,3.0,4.0,3.0,0.0,8.0,0.0,169,2.0,2.0,188.0,3.0,30,30,148.0,107.0,90.0,79.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,21410.0,116.0,0.0,0.0,2.0,2.0,1.0,0.0,None,0,0,None,None,None,None,None,None,None,None,None


In [9]:
import datetime

# There are two identical columns named match_id. I am removing the redundant match_id by it's index number.
#data.drop([data.columns[16]], axis=1, inplace=True)

# Setting the year column to an integer
data['match_year'] = data['match_year'].apply(lambda x: int(x) if pd.isnull(x) == False else x)

# Setting the match number to an integer
data['match_num'] = data['match_num'].apply(lambda x: int(x) if pd.isnull(x) == False else x)

# Having previously checked the value counts of match status - there are a number of matches listed 'In Progress'
# Since I know that isn't the case as all matches have been completed, I will update all matches listed as 
# "In Progress" to completed

# Creating a function to change all matches listed as 'In Progress' to 'Complete'
def status_of_match(row):
    if row == 'In Progress':
        return 'Complete'
    
# Applying status_of_match function  
data['match_status'] = data['match_status'].apply(status_of_match)


# I don't have much use for court_id's, since I have court names already. I will remove court ID's
data.drop('court_id', axis=1, inplace=True)

# I also don't have much use for player_id's since I already have player names, and I will refer to players using
# their names
data.drop(['player_1_id', 'player_2_is'], axis=1, inplace=True)


# Dropping p1_turning_point and p2_turning_point columns
data.drop(['p1_turning_point', 'p2_turning_point'], axis=1, inplace=True)

In [ ]:
# I found two values that were related to unknown points '0X' and '0Y' in the point_number column. I replaced these
# values with NaN values below
data['point_number'] = data['point_number'].replace('0X', np.nan).replace('0Y', np.nan)

# I also found some point numbers that ended with the letter D. I created a function below to find rows in
# point_number that end with the letter 'D', and replace those letters with ''

def removing_letters(row):
    row = str(row)
    if row.endswith('D'):
        row = row.replace('D', '')
    if row.endswith('X'):
        row = row.replace('X', '')
    if row.endswith('Y'):
        row = row.replace('Y', '')
    return row

# Applying the function above to the point_number feature
data['point_number'] = data['point_number'].apply(removing_letters)

# Dropping p1_turning_point and p2_turning_point columns
#data.drop(['p1_turning_point', 'p2_turning_point'], axis=1, inplace=True)

# Creating a list of remaining columns to convert to row values to floats for 

float_column_names = ['set_number', 'p1_games_won', 'p2_games_won', 'game_nuber', 'point_number', 'rally',
                     'p1_points_won', 'p2_points_won', 'serve_indicator', 'serve_number',
                    'rallycount', 'serve_speed_kmh', 'p1_momentum', 'p2_momentum', 'history', 
                      'speed_mph', 'p1_distance_run', 'p2_distance_run', 'p1_ace', 'p2_ace',
                     'p1_shot_winner', 'p2_shot_winner', 'p1_double_fault', 'p2_double_fault', 'p1_unforced_error',
                     'p2_unforced_error', 'p1_net_point', 'p2_net_point', 'p1_net_point_won', 'p2_net_point_won',
                     'p1_break_point', 'p2_break_point', 'p1_break_point_won', 'p2_break_point_won', 
                     'p1_first_serve_in', 'p2_first_serve_in', 'p1_first_serve_won', 'p2_first_serve_won',
                     'p1_second_serve_in', 'p2_second_serve_in', 'p1_second_serve_won', 'p2_second_serve_won',
                     'p1_forced_error', 'p2_forced_error', 'p1_break_point_missed', 'p2_break_point_missed',
                     'winner_fh', 'winner_bh', 'match_winner', 'set_winner', 'game_winner', 'point_winner', 
                      'point_server']

# Creating a for loop to covert all remaining float columns to numbers
for column in float_column_names:
    data[column] = data[column].apply(lambda x: float(x) if pd.isnull(x) == False else x)

In [ ]:
# Creating a list of winner features - set_winner, point_winner, etc.
winner_features = ['match_winner', 'set_winner', 'game_winner', 'point_winner', 'point_server']
    
# Remapping the values for the winner_features columns so it's easier to understand when converted to dummy variables  
for column in winner_features:
    data[column] = data[column].map({'1': 'Player 1', '2': 'Player 2'})

# Remapping the values for serve_width so it's easier to understand when converted to dummy variable
data['serve_width'] = data['servewidth'].map({'C': 'Down the T', 'W': 'Wide', 
                                           'BW': 'Body/ Wide', 'BC': 'Body/ Center', 'B': 'Body'})

# Remapping the values for serve_depth so it's easier to understand when converted to dummy variable
data['serve_depth'] = data['servedepth'].map({'CTL': 'Close to Line', 'NCTL': 'Not Close to Line'})

# Remapping the values for return_depth so it's easier to understand when converted to dummy variable
data['return_depth'] = data['returndepth'].map({'D': 'Deep', 'ND': 'Not Deep'})

data['serving_to'] = data['serving_to'].map({'d': 'Deuce', 'a': 'Ad'})

# Dropping the old servewidth, servedepth, and returndepth columns
data.drop(['servewidth', 'servedepth', 'returndepth'], axis=1, inplace=True)

# Creating a list of remaining columns to convert to row values to dummies for 

#dummy_column_names = ['serve_width', 'serve_depth', 'return_depth', 'serving_to']              



# Creating a for loop to covert all dummy columns to dummies
#for column in dummy_column_names:
#    data[column] = pd.get_dummies(data, columns=[column], drop_first=True)
         

In [ ]:
# I want to convert the elapsed_time to a time data type so I can calculate the length in time of points

# Converting data entries '0' for the start of the match to '00:00:00'
data['elapsed_time'] = data['elapsed_time'].replace('0', '00:00:00')

# I found an erroneuous entry where the hours where 25, and 26. I will update them upon further investigation of the 
# data

# Creating a new dataframe for only matches with this match_id
dn_mr = data[data['match_id'] == '2018-wimbledon-1306']

# Creating a new function to replace the hour values of 25, and 26 with 01, and 02 after investigating the data
def converting_time(row):
    if row[:2] == '25':
        row = '01' + row[2:]
    elif row[:2] == '26':
        row = '02' + row[2:]
    return row
  
# Applying the converting time function
dn_mr['elapsed_time'] = dn_mr['elapsed_time'].apply(converting_time)



# Adding the new values back into the original dataset 'data'
data.loc[data.index.isin(dn_mr.index), ['elapsed_time']] = dn_mr['elapsed_time']

data.dropna(subset=['elapsed_time'], inplace=True)

# Creating a function to convert any columns with 60 in the seconds column to 59
def converting_seconds(row):
    if row[-2:] == '60':
        row = row[:-2] + '59'
    return row

# Since I can't convert Null values to datetime, I calculated that about 23% of the entire dataset has 
# Null values for elapsed_time. Since there's nothing I can really do to fill this null data I have no choice but
# to drop it.
data['elapsed_time'] = data['elapsed_time'].apply(converting_seconds)

# Converting the 'elapsed_time' column to a time data type
# import datetime
import datetime

def convert_elapsed_time(row):
    if pd.isnull(row) == False:
        row = datetime.datetime.strptime(str(row), '%H:%M:%S')
    return row
        
data['elapsed_time'] = data['elapsed_time'].apply(convert_elapsed_time)

data.sort_values(by=['match_year', 'slam', 'match_id', 'elapsed_time'], inplace=True)

data['point_time'] = np.where(data.match_id == data.match_id.shift(), data.elapsed_time - 
                              data.elapsed_time.shift(), np.nan)
data['elapsed_time'] = data['elapsed_time'].apply(lambda x: pd.to_datetime(x).time())

In [ ]:
data.rename(columns={'game_nuber': 'game_number'}, inplace=True)

In [ ]:
data.info()

In [ ]:
data.head(10)

In [ ]:
# Filling all the na values of the match_year column with the first 4 characters in the match_id column since they
# each have the match year as the first 4 characters
df['match_year'].fillna(df.match_id.apply(lambda x: x[:4]), inplace=True)
df['match_year'] = df['match_year'].apply(lambda x: int(x))
df.match_year.isna().sum()

In [ ]:
# Updating NaN values for winner_fh
df['winner_fh'].fillna(df['winner_shot_type'].apply(lambda x: 1 if x == 'F' else 0), inplace=True)

# Updating NaN values for winner_bh
df['winner_bh'].fillna(df['winner_shot_type'].apply(lambda x: 1 if x == 'B' else 0), inplace=True)

In [16]:
data.to_csv('cleaned_data.csv')